# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:

cities_csv_path = "../WeatherPy/cities.csv"

# Import the data into a Pandas DataFrame
cities_df = pd.read_csv(cities_csv_path, index_col='Unnamed: 0')
cities_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hermanus,-34.4187,19.2345,16.67,86,100,0.39,ZA,1619373128
1,port alfred,-33.5906,26.8910,21.47,71,98,6.88,ZA,1619373427
2,bluff,-46.6000,168.3333,5.56,84,78,2.68,NZ,1619373110
3,dwarka,22.2394,68.9678,27.96,78,0,7.21,IN,1619373431
4,qaanaaq,77.4840,-69.3632,-0.15,73,80,1.61,GL,1619373342
...,...,...,...,...,...,...,...,...,...
470,hargeysa,9.5600,44.0650,21.98,69,88,3.83,SO,1619374058
471,vieux-habitants,16.0589,-61.7659,30.00,58,75,7.20,GP,1619374059
472,toamasina,-18.1667,49.3833,23.00,94,75,2.57,MG,1619374061
473,palmer,42.1584,-72.3287,12.00,76,90,2.57,US,1619374062


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:

gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into locations
locations = cities_df[["Lat", "Lng"]].astype(float)

# Store 'Humidity' as list
humidity = cities_df["Humidity"].astype(float)


In [32]:
fig = gmaps.figure(zoom_level=2, center=(16.276737, 1.482970))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:

# Drop cities with max temperature 80 degrees or higher
new_df = cities_df.loc[(cities_df["Max Temp"] < 80)]

# Drop cities with max teperature lower than 70 degrees
new_df = new_df.loc[(new_df["Max Temp"] > 70)]

# Drop cities with wind speed 10 mph or greater
new_df = new_df.loc[(new_df["Wind Speed"] < 10)]

# Drop cities with cloudiness greater than 0
new_df = new_df.loc[(new_df["Cloudiness"] == 0)]
new_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = new_df


hotel_df["Hotel Name"] = ""
hotel_df

# Set parameters to search for hotels within 5000 meters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    

    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change location each iteration while leaving origional params in place
    params["location"] = f"{lat},{lng}"
    
    # URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_name = requests.get(base_url, params=params)
    
    # Convert to json
    hotel_name = hotel_name.json()
    
    # try-except to skip any missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except:
        print("Missin field/result ... skipping")


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig = gmaps.figure(zoom_level=2, center=(16.276737, 1.482970))
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))